<a href="https://colab.research.google.com/github/ErSKS/Colab/blob/main/07_Transfer_Learning_Keras_Inceptionv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import Required Libraries

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import os
import sys
import glob

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

Using TensorFlow backend.


#### Configure Image width, height, number of epochs, batch size, fully connected layer neurons and number of layers to freeze on fine-tuning

In [ ]:
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 3
BAT_SIZE = 1
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172

#### Get number of files by searching directory recursively

In [ ]:
def get_nb_files(directory):
  
  if not os.path.exists(directory):
    return 0
  cnt = 0
  for r, dirs, files in os.walk(directory):
    for dr in dirs:
      cnt += len(glob.glob(os.path.join(r, dr + "/*")))
  return cnt

#### Freeze all layers of base inception model and compile the new model

In [ ]:
def setup_to_transfer_learn(model, base_model):
  
  for layer in base_model.layers:
    layer.trainable = False
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

####   Add last layer to the convnet
  Args: base_model: keras model excluding top, nb_classes: # of classes

  Returns: new keras model with last layer

In [ ]:
def add_new_last_layer(base_model, nb_classes):

  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
  predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
  model = Model(inputs=base_model.input, outputs=predictions)
  return model

####   Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
  note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
  Args:
    model: keras model

In [ ]:
def setup_to_finetune(model):

  for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
     layer.trainable = False
  for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
     layer.trainable = True
  model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

#### Use transfer learning and fine-tuning to train a network on a new dataset

In [ ]:
nb_train_samples = get_nb_files('../cash_project_large_db/data_20/train/')
nb_classes = len(glob.glob("../cash_project_large_db/data_20/train/*"))
nb_val_samples = get_nb_files('../cash_project_large_db/data_20/valid')
nb_epoch = int(3)
batch_size = int(32)

#### Prepare Data for Training

In [ ]:
train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
'../cash_project_large_db/data_20/train',
target_size=(IM_WIDTH, IM_HEIGHT),
batch_size=batch_size,
)

Found 40 images belonging to 2 classes.


#### Prepare data for testing

In [ ]:
test_datagen = ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)

validation_generator = test_datagen.flow_from_directory(
'../cash_project_large_db/data_20/valid',
target_size=(IM_WIDTH, IM_HEIGHT),
batch_size=batch_size,
)

Found 8 images belonging to 2 classes.


#### Load Base model (InceptionV3 without Top Layers)

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

#### Create a New Model with layers from base model and new top layers

In [ ]:
model = add_new_last_layer(base_model, nb_classes)

#### Setup Model for transfer learning 
By making base model layers non-trainable and only new top layers trainable, and compile the new model with RMSprop Optimizer

In [ ]:
setup_to_transfer_learn(model, base_model)

#### Train the model

In [ ]:
history_tl = model.fit_generator(
train_generator,
epochs=nb_epoch,
steps_per_epoch=nb_train_samples,
validation_data=validation_generator,
validation_steps=nb_val_samples)

Epoch 1/3
 2/40 [>.............................] - ETA: 423s - loss: 5.4297 - acc: 0.4219

#### Setup Model for finetune

In [ ]:
setup_to_finetune(model)

#### Again train the model with new configuration

In [ ]:
history_ft = model.fit_generator(
train_generator,
steps_per_epoch=nb_train_samples,
epochs=nb_epoch,
validation_data=validation_generator,
validation_steps=nb_val_samples)

Epoch 1/3
42/42 [==============================] - 919s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/3
42/42 [==============================] - 916s - loss: 8.0848 - acc: 0.4984 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/3
42/42 [==============================] - 957s - loss: 7.8787 - acc: 0.5112 - val_loss: 8.0590 - val_acc: 0.5000


#### After Training save the model for future Evaluation use

In [ ]:
model.save('inceptionv3-ft_2.model')